<a href="https://colab.research.google.com/github/dhrsajk9/Brain/blob/main/PrefrontalCortex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
# Load the dataset
file_path = "/content/drive/MyDrive/Brain/Prefrontal_Cortex/train.csv"
df = pd.read_csv(file_path)

# Display basic info and the first few rows
df_info = df.info()
df_head = df.head()
df_description = df.describe(include="all")

df_info, df_head, df_description

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52002 entries, 0 to 52001
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  52002 non-null  object
 1   input        20658 non-null  object
 2   output       52002 non-null  object
 3   text         52002 non-null  object
dtypes: object(4)
memory usage: 1.6+ MB


(None,
                                          instruction input  \
 0               Give three tips for staying healthy.   NaN   
 1                 What are the three primary colors?   NaN   
 2                 Describe the structure of an atom.   NaN   
 3                   How can we reduce air pollution?   NaN   
 4  Describe a time when you had to make a difficu...   NaN   
 
                                               output  \
 0  1. Eat a balanced and nutritious diet: Make su...   
 1  The three primary colors are red, blue, and ye...   
 2  An atom is the basic building block of all mat...   
 3  There are several ways to reduce air pollution...   
 4  As an AI assistant, I do not have my own perso...   
 
                                                 text  
 0  Below is an instruction that describes a task....  
 1  Below is an instruction that describes a task....  
 2  Below is an instruction that describes a task....  
 3  Below is an instruction that describes a 

In [ ]:
# Use a fraction of the dataset (e.g., 60%)
#fraction = 0.6
#df_small = df.sample(frac=fraction, random_state=42).reset_index(drop=True)


In [ ]:
import random

# Function to create reasoning-enhanced text
def create_reasoning_prompt(row):
    instruction = row['instruction']
    input_text = row['input'] if pd.notna(row['input']) else "None"
    output = row['output']

    reasoning_steps = [
        "Step 1: Understand the instruction clearly.",
        "Step 2: Consider relevant context or input.",
        "Step 3: Evaluate possible outcomes and select the most logical response."
    ]
    reasoning = "\n".join(reasoning_steps)

    # Formatting target to match desired structure
    prompt = f"Instruction: {instruction}\nInput: {input_text}\nReasoning:\n{reasoning}\n\nFinal Decision:"
    target = f"Final Decision: {output.strip()}"

    return {"input_text": prompt, "target_text": target}

# Apply reasoning format to all rows
formatted_data = df.apply(create_reasoning_prompt, axis=1)
formatted_df = pd.DataFrame(formatted_data.tolist())

# Save to CSV for training
formatted_dataset_path = "D:\Projects\Brain\Prefrontal Cortex Data\\train2.csv"
formatted_df.to_csv(formatted_dataset_path, index=False)

formatted_df.head()

,input_text,target_text
0,Instruction: Give three tips for staying healt...,Final Decision: 1. Eat a balanced and nutritio...
1,Instruction: What are the three primary colors...,Final Decision: The three primary colors are r...
2,Instruction: Describe the structure of an atom...,Final Decision: An atom is the basic building ...
3,Instruction: How can we reduce air pollution?\...,Final Decision: There are several ways to redu...
4,Instruction: Describe a time when you had to m...,"Final Decision: As an AI assistant, I do not h..."


In [ ]:
print("Columns:", formatted_df.columns)

Columns: Index(['input_text', 'target_text'], dtype='object')


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
# Split into train and test
train_df, test_df = train_test_split(formatted_df, test_size=0.2, random_state=42)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Custom dataset class
class DecisionDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=512):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        input_text = str(self.data.iloc[index]["input_text"])
        target_text = str(self.data.iloc[index]["target_text"])

        input_encoding = self.tokenizer(
            input_text, padding="max_length", truncation=True, max_length=self.max_len, return_tensors="pt"
        )
        target_encoding = self.tokenizer(
            target_text, padding="max_length", truncation=True, max_length=self.max_len, return_tensors="pt"
        )

        labels = target_encoding["input_ids"]
        labels[labels == tokenizer.pad_token_id] = -100  # For loss masking

        return {
            "input_ids": input_encoding["input_ids"].squeeze(),
            "attention_mask": input_encoding["attention_mask"].squeeze(),
            "labels": labels.squeeze()
        }


In [ ]:
# Create datasets and loaders
train_dataset = DecisionDataset(train_df, tokenizer)
test_dataset = DecisionDataset(test_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=6, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=6)


In [ ]:
# Evaluation function
def evaluate_model(model, dataloader):
    model.eval()
    predictions = []
    references = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=100)

            decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            predictions.extend(decoded_preds)
            references.extend(decoded_labels)

    return predictions, references

# After training, call:
# preds, targets = evaluate_model(model, test_loader)

In [ ]:
from torch.optim import AdamW

In [ ]:
#Approx 3 hours
optimizer = AdamW(model.parameters(), lr=5e-5)
epochs = 3  # You can change this
model.train()

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    total_loss = 0
    loop = tqdm(train_loader, leave=True)

    for batch in loop:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()

        optimizer.step()

        total_loss += loss.item()
        loop.set_description(f"Epoch {epoch + 1}")
        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"Average Loss: {avg_loss:.4f}")

Epoch 1/3


Epoch 1: 100%|██████████| 6934/6934 [55:12<00:00,  2.09it/s, loss=2.57]


Average Loss: 3.2582
Epoch 2/3


Epoch 2: 100%|██████████| 6934/6934 [55:13<00:00,  2.09it/s, loss=2.99]


Average Loss: 3.0549
Epoch 3/3


Epoch 3: 100%|██████████| 6934/6934 [55:13<00:00,  2.09it/s, loss=1.65]

Average Loss: 2.9650


In [ ]:
from transformers import GenerationConfig
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Directory to save your model
save_path = "my_trained_model"

# Save model weights as .safetensors
model.save_pretrained(save_path, safe_serialization=True)
tokenizer.save_pretrained(save_path)

# (Optional) Save generation config
generation_config = GenerationConfig(
    max_length=128,
    temperature=0.9,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=1
)
generation_config.save_pretrained(save_path)

print(f"Model saved to '{save_path}' ✅")


Model saved to 'my_trained_model' ✅


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import T5Tokenizer, T5ForConditionalGeneration, GenerationConfig

# Directory where your model was saved
save_path = "my_trained_model"

# Load the saved model and tokenizer
tokenizer = T5Tokenizer.from_pretrained(save_path)
model = T5ForConditionalGeneration.from_pretrained(save_path)
model.to(device)  # Move model to your device (CPU or GPU)

# Evaluation function (same as before)
def evaluate_model(model, dataloader):
    # ... (same code as provided in the previous response) ...

# Load the test dataset and create the DataLoader
test_dataset = DecisionDataset(test_df, tokenizer)  # Assuming test_df is available
test_loader = DataLoader(test_dataset, batch_size=6)

# Evaluate the loaded model
preds, targets = evaluate_model(model, test_loader)

In [ ]:
def decide(instruction, input_text):
    prompt = f"""Instruction: {instruction}
Input: {input_text}
Reasoning:
Step 1: Understand the instruction clearly.
Step 2: Consider relevant context or input.
Step 3: Evaluate possible outcomes and select the most logical response.

Final Decision:"""

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    output = model.generate(
        input_ids,
        max_length=128,
        temperature=0.9,
        top_p=0.95,
        do_sample=True,
        num_return_sequences=1
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)





In [ ]:
response = decide(
    "I am going on a ride",
    "How should I prepare?"
)

# Just in case the model repeats input, strip it out
decision = response.split("Final Decision:")[-1].strip()
print("🧠 Model's Decision:", decision)




🧠 Model's Decision: Yes, I am going on a ride. However, I can tell you to take a ride and do some preparation. Here are some tips you can use: 1. Have a warm meal: Always be prepared for the ride and your friends. Bring your friends, family, friends, and family to the ride. 2. Start by preparing your friends and family to enjoy the ride. 3. Do some camping and camping activities. 5. Plan out your activities and activities, and use your boarding gear. 7. If necessary, take the time for the ride. 8. Do you have a plan, such as


In [ ]:
response = decide(
    "I am hungry",
    "What do I do?"
)

# Just in case the model repeats input, strip it out
decision = response.split("Final Decision:")[-1].strip()
print("🧠 Model's Decision:", decision)

🧠 Model's Decision: I'm hungry as a foodie, and I have an appetite, just asking for food. To find something that would need to be weighed, it is important to take a few minutes or week to determine the most important person. It's essential that you have a healthy body and everything is well-earned for life. These snacks can be eaten, such as a small sandbox, a cup of hot water, and some snacks. The following options are: 1. Get a great food: I do a lot of eggs, fruits, and other foods,


In [ ]:
response = decide(
    "Do you want some food?",
    "It is good."
)

# Just in case the model repeats input, strip it out
decision = response.split("Final Decision:")[-1].strip()
print("🧠 Model's Decision:", decision)

🧠 Model's Decision: Yes, you want to get some food: It's good.
